Drawn from https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/

In [2]:
# Small LSTM Network to Generate Text for Alice in Wonderland
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from datetime import datetime
import os

In [4]:
#"http://www.gutenberg.org/cache/epub/11/pg11.txt"
#scp -i /c/blah/.ssh/blah.pem /d/blah/alice_in_wonderland_11-0.txt ubuntu@blah:~/blah/

In [5]:
# load ascii text and covert to lowercase
filename = "alice_in_wonderland_11-0.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

In [7]:
print type(raw_text), len(raw_text)

<type 'str'> 173595


In [8]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print "Total Characters: ", n_chars
print "Total Vocab: ", n_vocab


Total Characters:  173595
Total Vocab:  66


In [9]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print "Total Characters: ", n_chars
print "Total Vocab: ", n_vocab


Total Characters:  173595
Total Vocab:  66


In [11]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print "Total Patterns: ", n_patterns


Total Patterns:  173495


In [12]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
print X.shape

(173495, 100, 1)


In [16]:
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
print type(y), y.shape
print y[0]

<type 'numpy.ndarray'> (173495, 65)
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


In [20]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
print model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lstm_4 (LSTM)                    (None, 256)           264192      lstm_input_4[0][0]               
____________________________________________________________________________________________________
dropout_4 (Dropout)              (None, 256)           0           lstm_4[0][0]                     
____________________________________________________________________________________________________
dense_4 (Dense)                  (None, 65)            16705       dropout_4[0][0]                  
Total params: 280,897
Trainable params: 280,897
Non-trainable params: 0
____________________________________________________________________________________________________
None


In [21]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
#https://keras.io/callbacks/
#Save the model after every epoch.
callbacks_list = [checkpoint]


In [23]:
# fit the model
startTime= datetime.now()
print ("started at ", startTime)
model.fit(X, y, nb_epoch=20, batch_size=128, callbacks=callbacks_list)
timeElapsed=datetime.now()-startTime
print('Time elapsed (hh:mm:ss.ms) {}'.format(timeElapsed))


('started at ', datetime.datetime(2017, 11, 21, 22, 58, 37, 567207))
Epoch 1/20
173495/173495 [==============================] - 197s - loss: 2.9847   
Epoch 2/20
173495/173495 [==============================] - 196s - loss: 2.6913   
Epoch 3/20
173495/173495 [==============================] - 193s - loss: 2.5831   
Epoch 4/20
173495/173495 [==============================] - 190s - loss: 2.5150   
Epoch 5/20
173495/173495 [==============================] - 189s - loss: 2.4583   
Epoch 6/20
173495/173495 [==============================] - 189s - loss: 2.4011   
Epoch 7/20
173495/173495 [==============================] - 189s - loss: 2.3566   
Epoch 8/20
173495/173495 [==============================] - 189s - loss: 2.3150   
Epoch 9/20
173495/173495 [==============================] - 189s - loss: 2.2795   
Epoch 10/20
173495/173495 [==============================] - 189s - loss: 2.2465   
Epoch 11/20
173495/173495 [==============================] - 189s - loss: 2.2157   
Epoch 12/20
1734